# Homework 6

In this HW, we will study the pay gap between men and women who have jobs in San Francisco.  We will use the following two csv files to accomplish this task.

- Salaries.csv : contains salaries for over 100K employees in SF from 2011-2014.
- Names.csv :  contains baby names from 1980 to 2014 along with counts of how many times the given baby name was used.

We would like to find the average salary of men and women over all jobs from 2011 to 2014.  The problem, however, is that the Salaries.csv does not contain each person's sex.  Futher, there are many names that are unisex, like Alex.  Since we have counts in the file Names.csv, we can use a majority vote to label the sex of each name in Names.csv. 

You will be asked to write a series of functions to implement this task. **YOU SHOULD NOT BE USING FOR LOOPS ANYWHERE IS THIS ASSIGMENT. IF YOUR CODE IS TAKING MORE THAN 30 SECONDS TO RUN, THEN YOU HAVE DONE SOMETHING WRONG.**




The following function reads in the salary and the names data as pandas dataframes and return a list which contains these two dataframes. 

In [17]:
#Place your import here
import pandas as pd
import numpy as np

def ReadData():

    df_salaries = pd.read_csv("Data/Salaries.csv")
    df_names = pd.read_csv("Data/Names.csv")
    
    
    return [df_salaries, df_names]

In [18]:
[df_salaries, df_names] = ReadData()
df_salaries.head()

,EmployeeName,JobTitle,Total_Pay,Year
0,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,567595.43,2011
1,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),538909.28,2011
2,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),335279.91,2011
3,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,332343.61,2011
4,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",326373.19,2011


In [19]:
df_names.head()

,Name,Year,Sex,Count
0,Jessica,1990,F,46466
1,Ashley,1990,F,45549
2,Brittany,1990,F,36535
3,Amanda,1990,F,34406
4,Samantha,1990,F,25864


In [20]:
parsed_names = df_salaries.EmployeeName.str.split(" ")
parsed_names[0][0]

'NATHANIEL'

## Problem 1 -  Get name counts

The following function **Get_Counts** will take df_names as an input. It will then output a dataframe called df_count where the index has a entry for each name that appears in df_names, and there are two columns "male_count" and "female_count" indicating the number of times (sum of the "Count" column) the particular name was ever used for a boy and girl respectively. Hint: One way to do this is to use the unstack method after the groupby.



In [21]:
def Get_Counts(df_names):
    
    
    #Initialize empty dictionaries for names
    df_count = None
    
    # YOUR CODE HERE
    count = df_names.drop(columns='Year').groupby(["Name", "Sex"]).sum().unstack(level=-1).reset_index()

    count.columns = count.columns.map(''.join)
    mapping = {count.columns[0]: "Name", count.columns[1]: "female_count", count.columns[2]: "male_count"}
    count = count.rename(columns=mapping).fillna(0).set_index('Name')
    
    
    return count

In [22]:
[df_salaries, df_names] = ReadData()
df_count = Get_Counts(df_names)
assert df_count.shape[0] == 73402
assert df_count.loc["Aamir", "female_count"] == 6
assert df_count.loc["Alex", "female_count"] == 6378

## Problem 2 -  Get_Sex

This function takes in df_counts from the function Get_Counts and df_salaries and it returns a data called df_combined that has the following three columns:
    - Total_Pay (from df_salaries)
    - Year (from df_salaries)
    - Sex (to be created by you)
    
There will a row in df_combined for each person (row) in df_salaries whose first name appears in df_counts, i.e. we have either/both a female_count and male_count for this first name. If the first name does not appear in df_counts, then you can discard this person.  The way we will guess the sex of each person in df_salaries is to first pull out their first name from the EmployeeName column, and then check the male and female counts in df_couts of this first name, and choose the sex with the higher count. If there is a tie, we will choose female. Use "M" to indicate male, and "F" to indicate "female". 

In [23]:
def Get_Sex(df_salaries, df_count):
    
    
    df_combined = None
    
    df_salaries['Name'] = df_salaries['EmployeeName'].apply(lambda x: x.split(' ')[0])
    df_salaries['Name'] = df_salaries['Name'].str.lower()
    df_temp = df_count.reset_index()
    df_temp['Name'] = df_temp['Name'].str.lower()
    merged = pd.merge(df_temp, df_salaries, on='Name')
    merged['Sex'] = merged.apply(lambda x: 'F' if (x.female_count >= x.male_count) else 'M', axis=1)

    df_combined = merged[['Total_Pay', 'Year', 'Sex']].copy() 
    
    return df_combined

In [24]:
[df_salaries, df_names] = ReadData()
df_count = Get_Counts(df_names)
df_combined = Get_Sex(df_salaries, df_count)
assert df_combined.shape[0] == 106718

In [25]:
assert df_combined.Sex.value_counts()["M"] == 62036

## Problem 3 -  Compute the Average Salary

In this problem, you will implement a function called **ComputeAvgSalary**. This function takes in df_combined and one of the following four years 2011, 2012, 2013, 2014, and it returns the ratio of avg_male_salary/avg_female_salary for all people in df_combined restricted to the specified year.

In [26]:
def ComputeAvgSalary(df_combined, year):

    ratio = None
    
    avg_male_salary = df_combined['Total_Pay'].loc[(df_combined['Year'] == year) & (df_combined['Sex'] == 'M')].mean()
    avg_female_salary = df_combined['Total_Pay'].loc[(df_combined['Year'] == year) & (df_combined['Sex'] == 'F')].mean()
    ratio = avg_male_salary/avg_female_salary
    
    return ratio

In [27]:
[df_salaries, df_names] = ReadData()
df_count = Get_Counts(df_names)
df_combined = Get_Sex(df_salaries, df_count)
assert np.isclose(ComputeAvgSalary(df_combined, 2011),1.173526)

In [28]:
assert np.isclose(ComputeAvgSalary(df_combined, 2014),1.1444)